In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
from os.path import join
import re

# Funções Úteis

In [21]:
def match_file_names(
    pattern: str,
    path: str):

    matched_files = []

    for root, _, files in os.walk(path):
        for file in files:
            if file.lower().endswith(".csv") and pattern.search(file):
                matched_files.append(os.path.join(root, file))

    return matched_files

In [40]:
def read_files(
    file_names: list[str],
    encoding: str = "utf8",
    delimiter: str = ","):

    df_list = []
    for file_name in file_names:

        with open(file_name, "r", encoding="latin-1") as file:
            lines = file.readlines()
        
        header_index = next(i for i, line in enumerate(lines) if line.startswith("Data"))

        try:
            df = pd.read_csv(
                file_name, 
                skiprows=header_index,
                encoding=encoding, 
                delimiter=delimiter)
            df = df.drop(columns=["Unnamed: 19"])
            df_list.append(df)
        except Exception as e:
            print(f"Exception {e} on {file_name}")

        
    return df_list

In [153]:
def replace_comma(value):

    if isinstance(value, str): 
        return pd.to_numeric(value.replace(",", "."), errors='coerce') 
    
    return value 

# Coleta de Dados

In [4]:
MODULE_PATH = Path().absolute().parent
DATA_PATH = join(MODULE_PATH, "sources")

In [202]:
pattern = re.compile(r"BELO[\s_\-]?HORIZONTE|PAMPULHA", re.IGNORECASE)
matched_files = match_file_names(pattern=pattern, path=DATA_PATH)

In [203]:
df_list = read_files(file_names=matched_files, encoding="latin-1", delimiter=";")

# Pré-processamento

In [204]:
for df in df_list:
    df["Hora UTC"] = df["Hora UTC"].str.replace(" UTC", "")
    df["Datetime"] = pd.to_datetime(df["Data"] + " " + df["Hora UTC"], format="%Y/%m/%d %H%M")

    df.rename(columns={"RADIACAO GLOBAL (KJ/m²)": "RADIACAO GLOBAL (Kj/m²)"}, inplace=True)

    for col in [col for col in df.columns if col not in ["Data", "Hora UTC", "Datetime"]]:
        df[col] = df[col].apply(replace_comma)

In [206]:
df_years_dict = {}

i = 0
for first_file_name, first_df in zip(matched_files, df_list):
    first_file_year = first_file_name.split("\\")[-2]

    common_year_df_list = [first_df]
    for second_file_name, second_df in zip(matched_files[i+1:], df_list[i+1:]):
        second_file_year = second_file_name.split("\\")[-2]

        if first_file_year == second_file_year:
            common_year_df_list.append(second_df)

    if first_file_year not in df_years_dict.keys():
        df_years_dict[first_file_year] = common_year_df_list
    i+=1

In [207]:
combined_df_by_year_dict = {}
for year, year_df_list in df_years_dict.items():

    year_df_list = [df.set_index("Datetime").asfreq("h").reset_index() for df in year_df_list]

    combined_df = pd.concat(year_df_list)
    combined_df = combined_df.drop(columns=["Data", "Hora UTC"])
    result = combined_df.groupby("Datetime").mean().reset_index()

    combined_df_by_year_dict[year] = result

In [208]:
complete_df = pd.concat(list(combined_df_by_year_dict.values())).set_index("Datetime").asfreq("h")

In [209]:
complete_df

,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)"
Datetime,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,0.0,899.80,899.80,899.15,NaN,20.20,19.10,20.85,19.60,19.10,17.60,88.0,81.0,86.0,95.0,4.70,2.85
2019-01-01 01:00:00,0.0,900.35,900.35,899.80,NaN,19.85,18.50,20.25,19.75,19.00,18.40,86.0,84.0,85.0,105.0,5.40,3.40
2019-01-01 02:00:00,0.0,900.45,900.55,900.30,NaN,19.65,18.30,19.90,19.60,18.50,18.20,86.0,85.0,86.0,104.0,5.45,2.25
2019-01-01 03:00:00,0.0,899.75,900.45,899.75,NaN,19.50,17.90,19.75,19.35,18.30,17.90,87.0,85.0,86.0,101.0,4.10,1.55
2019-01-01 04:00:00,0.0,899.80,899.85,899.55,NaN,19.45,18.10,19.70,19.05,18.20,17.60,89.0,86.0,88.0,84.5,3.95,1.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 19:00:00,0.0,897.70,898.30,897.50,1551.45,24.55,18.75,27.90,24.45,19.80,18.00,70.0,59.0,70.0,92.5,9.05,3.70
2023-12-31 20:00:00,0.1,897.45,897.70,897.40,301.15,23.00,18.55,24.70,22.95,18.90,18.00,77.0,68.0,76.5,110.0,8.20,4.80
2023-12-31 21:00:00,3.4,897.95,897.95,897.35,49.85,20.90,18.80,23.05,20.90,19.05,18.45,88.0,76.5,88.0,140.5,10.70,5.50
